In [1]:
import textract
text = textract.process("path/to/file.extension")

ModuleNotFoundError: No module named 'textract'

In [2]:
!pip install textract

  Using cached textract-1.6.5-py3-none-any.whl (23 kB)
  Using cached six-1.12.0-py2.py3-none-any.whl (10 kB)
  Using cached python-pptx-0.6.21.tar.gz (10.1 MB)
  Preparing metadata (setup.py) ... done
  Using cached extract_msg-0.28.7-py2.py3-none-any.whl (69 kB)
  Using cached SpeechRecognition-3.8.1-py2.py3-none-any.whl (32.8 MB)
  Using cached pdfminer.six-20191110-py2.py3-none-any.whl (5.6 MB)
  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
  Using cached beautifulsoup4-4.8.2-py3-none-any.whl (106 kB)
  Using cached argcomplete-1.10.3-py2.py3-none-any.whl (36 kB)
  Using cached docx2txt-0.8.tar.gz (2.8 kB)
  Preparing metadata (setup.py) ... done
  Using cached xlrd-1.2.0-py2.py3-none-any.whl (103 kB)
  Using cached pycryptodome-3.15.0-cp35-abi3-macosx_10_9_x86_64.whl (1.6 MB)
  Using cached ebcdic-1.1.1-py2.py3-none-any.whl (128 kB)
  Using cached IMAPClient-2.1.0-py2.py3-none-any.whl (73 kB)
  Using cached tzlocal-4.2-py3-none-any.whl (19 kB)
  Using cached compressed

In [ ]:
"""
Created on March 2018


@author: 
	
▒█▀▀█ █▀▀█ █▀▀▄ █▀▀ ▒█▀▀█ █░░█ ▀▀█▀▀ █▀▀ 
▒█░░░ █░░█ █░░█ █▀▀ ▒█▀▀▄ █▄▄█ ░░█░░ █▀▀ 
▒█▄▄█ ▀▀▀▀ ▀▀▀░ ▀▀▀ ▒█▄▄█ ▄▄▄█ ░░▀░░ ▀▀▀ 


"""

#################################################################################################################################################




"""
●   A web app to help employers by analysing resumes and CVs, surfacing candidates that best match the position and filtering out those who don't.
●   Used recommendation engine techniques such as KNN, content based filtering for fuzzy matching job description with multiple resumes.

Prerequisites

    Gensim
    Numpy==1.11.3
    Pandas
    Sklearn
    Dash
    Antiwords
    autocorrect




        To Run this code: 
            # python app.py

	And open URL localhost:5000


"""

import glob
import os
import warnings
import textract
import requests
from flask import (Flask,session, g, json, Blueprint,flash, jsonify, redirect, render_template, request,
                   url_for, send_from_directory)
from gensim.summarization import summarize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from werkzeug import secure_filename

import pdf2txt as pdf
import PyPDF2

import screen
import search
import hashlib

warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

app = Flask(__name__)

app.config.from_object(__name__) # load config from this file , flaskr.py

# Load default config and override config from an environment variable
app.config.update(dict(
    USERNAME='admin',
    PASSWORD='7b4d7a208a333b46acdc9da159e5be7a',
    SECRET_KEY='development key',
))


app.config['UPLOAD_FOLDER'] = 'Original_Resumes/'
app.config['ALLOWED_EXTENSIONS'] = set(['txt', 'pdf', 'png', 'jpg', 'jpeg', 'gif'])

class jd:
    def __init__(self, name):
        self.name = name

def getfilepath(loc):
    temp = str(loc).split('\\')
    return temp[-1]
    



@app.route('/login', methods=['GET', 'POST'])
def login():
    error = None
    if request.method == 'POST':
        if request.form['username'] != app.config['USERNAME']:
            error = 'Invalid username'
        elif app.config['PASSWORD'] != hashlib.md5(request.form['password'].encode('utf-8')).hexdigest():
            error = 'Invalid password'
        else:
            session['logged_in'] = True
            flash('You were logged in')
            return redirect(url_for('home'))
    return render_template('login.html', error=error)

@app.route('/logout')
def logout():
    session.pop('logged_in', None)
    flash('You were logged out')
    return redirect(url_for('home'))


@app.route('/')
def home():
    x = []
    for file in glob.glob("./Job_Description/*.txt"):
        res = jd(file)
        x.append(jd(getfilepath(file)))
    print(x)
    return render_template('index.html', results = x)




@app.route('/results', methods=['GET', 'POST'])
def res():
    if request.method == 'POST':
        jobfile = request.form['des']
        print(jobfile)
        flask_return = screen.res(jobfile)
        
        print(flask_return)
        return render_template('result.html', results = flask_return)



@app.route('/resultscreen' ,  methods = ['POST', 'GET'])
def resultscreen():
    if request.method == 'POST':
        jobfile = request.form.get('Name')
        print(jobfile)
        flask_return = screen.res(jobfile)
        return render_template('result.html', results = flask_return)



@app.route('/resultsearch' ,methods = ['POST', 'GET'])
def resultsearch():
    if request.method == 'POST':
        search_st = request.form.get('Name')
        print(search_st)
    result = search.res(search_st)
    # return result
    return render_template('result.html', results = result)


@app.route('/Original_Resume/<path:filename>')
def custom_static(filename):
    return send_from_directory('./Original_Resumes', filename)



if __name__ == '__main__':
   # app.run(debug = True) 
    # app.run('127.0.0.1' , 5000 , debug=True)
    app.run('0.0.0.0' , 5000 , debug=True , threaded=True)